In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

import implicit
import lightfm
import warnings
from validation_prepare import *
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [3]:
actions = pd.read_pickle(PREPARED_PATH+'actions_one_table.pkl')

actions.sort_index(inplace = True) # На всякий случай, иначе деление не будет работать

actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [4]:
idx = get_train_test(actions)

6558458
2186152
2186153


In [5]:
actions.iloc[idx[0]].index.get_level_values(2).max()

43362401.96226887

In [6]:
actions.iloc[idx[1]].index.get_level_values(2).min()

43362401.97085199

In [7]:
actions.iloc[idx[1]].index.get_level_values(2).max()

43828341.47903843

In [8]:
actions.iloc[idx[2]].index.get_level_values(2).min()

43828341.48519237

In [9]:
actions.consumption_mode.value_counts()

S    8296227
P     873834
R     472951
Name: consumption_mode, dtype: int64

In [10]:
actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [11]:
# Вроде не пересекается.
train,test,valid = actions.iloc[idx[0]],actions.iloc[idx[1]],actions.iloc[idx[2]]

In [12]:
%time 
dur_being_train,dur_films_train,watch_actions_train,target_train = get_target(train)
dur_being_test,dur_films_test,watch_actions_test,target_test = get_target(test)
dur_being_valid,dur_films_valid,watch_actions_valid,target_valid = get_target(valid)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.86 µs


In [13]:
# Получили фичи для фильмов
import pickle
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)
movie_match_columns = {i:ii for ii,i in enumerate(match_columns)}
movie_columns_match = {ii:i for ii,i in enumerate(match_columns)}

with open(PREPARED_PATH+'bag_of_attr_movie.pkl','rb') as f:
    bag_of_attr = pickle.load(f)

In [14]:
fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,)
fe.fit(train)

100%|██████████| 389188/389188 [03:26<00:00, 1880.74it/s]


(389188, 29221) 389188 29221


In [15]:
train_ = fe.transform(train)
test_ = fe.transform(test)

train_.shape,test_.shape 

cfe = ColdFeatureExtractor(fe)

cfe.fit(train)

max(cfe.train_movie_cols)

# sp.csc_matrix(cfe.fitted_FE.movie_attr_matrix)[cfe.train_movie_rows,cfe.train_movie_cols]



train_res = cfe.transform(test)


print(train_res['train_user'].shape,train_res['test_user'].shape,train_res['new_test_user'].shape)

print(train_res['train_movie'].shape,train_res['test_movie'].shape)

8015 389188
(286374, 1) (281788, 3)
(389188, 8015)
8015 389188
(85260, 1) (63113, 3)
(389188, 8015)


100%|██████████| 35799/35799 [00:14<00:00, 2516.89it/s]


((8015, 29232), (251, 29232))

In [16]:
# def fit_lightfm(train,item_features=None,seed = 0)
seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)


# model.fit(train_matrix,user_features = train_user,item_features = element_matrix,epochs = epochs,num_threads = num_threads,
#          verbose = True)


In [ ]:
tr = old_old_tr_te_1['train_interactions']
te = old_old_tr_te_1['test_interactions']
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,epochs = epochs,num_threads = num_threads,
         verbose = True)

In [ ]:
te.shape

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, te ,tr, k = 20,num_threads=4).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
new_old_tr_te_2.keys()

In [ ]:
tr = new_old_tr_te_2['train_interactions']
te = new_old_tr_te_2['test_interactions']
uf = new_old_tr_te_2['user_features_train']
tf = new_old_tr_te_2['user_features_test']
print(tr.shape,te.shape,uf.shape,tf.shape)
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,user_features=uf,epochs = epochs,num_threads =num_threads,
         verbose = True)

In [ ]:
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
tf = csr_matrix(tf)

In [ ]:
p = model.predict(0,np.arange(tr.shape[1]),user_features=tf[0,:],num_threads=num_threads)

In [ ]:
plt.hist(p)

In [ ]:
test_user[0,:]

In [ ]:
a = model.predict(0,np.arange(train_matrix.shape[1]),user_features=test_user[0,:],num_threads=num_threads)
len(a)

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, train_matrix,user_features = train_user,item_features = element_matrix ,num_threads=num_threads).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
model.user_feature_map

In [ ]:
def get_answer(test_users_dict,test):
    pass